In [8]:
import os
import cv2
import torch
from torchvision import transforms
from prototypical_net import ConvNet
from PIL import Image
import random
from detect_object_multiscale import detect_object_multiscale

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
patch_sizes=[84, 96, 112, 128]
stride=20
extensions=(".jpg", ".jpeg", ".png")
# Load model
model = ConvNet().to(device)
model.load_state_dict(torch.load("model.pth"))
model.eval()


ConvNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1

### Transforms

In [10]:
# Transforms
transform = transforms.Compose([
    transforms.Resize((84, 84)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3),
])

In [11]:
workspace_path = "images/workspace-images/workspace.png"
target_class = "apple"  # Change this to the desired class
support_dir = os.path.join("images/train-images", target_class)
support_path = random.choice([
    os.path.join(support_dir, f)
    for f in os.listdir(support_dir)
    if f.lower().endswith(extensions)
])

# Pick one random image from the class folder
sample_img_path = random.choice([
    os.path.join(support_dir, f) for f in os.listdir(support_dir) if f.endswith(extensions)
])
sample_img = Image.open(sample_img_path).convert("RGB")
sample_tensor = transform(sample_img).unsqueeze(0).to(device)
sample_embedding = model(sample_tensor)

# Load and process workspace image
workspace = cv2.imread(workspace_path)
workspace = cv2.cvtColor(workspace, cv2.COLOR_BGR2RGB)

In [12]:
result = detect_object_multiscale(
    model=model,
    support_path=support_path,
    workspace_path=workspace_path,
    patch_sizes=patch_sizes,
    stride=stride,
    device=device,
    distance=3,
    threshold=0.004,
    confidence=0.50
)
result
# Print the results
# print(f"Best match at: {result['location']}, Distance: {result['distance']:.4f}, Patch Size: {result['patch_size']}, Confidence: {result['confidence']:.4f}")


[{'location': (708, 408),
  'distance': 0.24623575806617737,
  'patch_size': 96,
  'confidence': 0.8024163915435479},
 {'location': (322, 442),
  'distance': 0.2586000859737396,
  'patch_size': 84,
  'confidence': 0.7945335544978381},
 {'location': (142, 82),
  'distance': 0.27765053510665894,
  'patch_size': 84,
  'confidence': 0.7826866365430039},
 {'location': (228, 208),
  'distance': 0.29757624864578247,
  'patch_size': 96,
  'confidence': 0.7706676205299315},
 {'location': (328, 68),
  'distance': 0.3331809639930725,
  'patch_size': 96,
  'confidence': 0.7500857175494415}]

In [13]:
print(f"Number of detected objects: {len(result)}")

Number of detected objects: 5


In [14]:
if result == []:
    print("Object not detected on the workspace.")
else:
    for loc in result:
        cv2.circle(workspace, loc['location'], 5, (255, 250, 0), 2)
    cv2.imwrite("detected.png", cv2.cvtColor(workspace, cv2.COLOR_RGB2BGR))